In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data\Eta_vs_Theta'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\Eta_vs_Theta'

### ssro calibrations
day = '20160726_' # for ssro calibration
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

Purify_analysis = pa.purify_analysis('Eta_v_Theta',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

msmt_days_unshifted = ['20160726']
msmt_days_shifted = ['20160824']

ms0_percents = [10,15,25,35,50,75]
real_theta_vals = np.flipud((0.5-np.array([0.153355,0.2525,0.2827,0.323754,0.362197,0.3865]))
)*np.pi
real_pop_vals = np.sin(real_theta_vals)**2
TPQI_vis = 0.725#0.725
TPQI_vis_u = 0.03
%matplotlib inline

In [ ]:
fidelities = []
fidelities_u = []
fidelities_mixed = []
fidelities_mixed_u = []
val00 = []
val11 = []

for ms0_percent in ms0_percents:
    if ms0_percent != 35:
        string = 'eta_'+str(ms0_percent)+'percent'
        st_start = 3055e3#2763e3
    else:
        string = 'ZZ_'+str(ms0_percent)+'percent'
        st_start = 3057e3#2763e3
        
    Purify_analysis.get_tstamps_and_offsets(unshifted_days = msmt_days_unshifted, shifted_days = msmt_days_shifted, contains = string)
    Purify_analysis.load_raw_data()
    
    ##### Set filters:
    st_start = 3055e3#2763e3
    st_len       = 40e3 #50 ns
    st_len_w2    = 40e3
    
    Purify_analysis.apply_temporal_filters_to_prefiltered_data(st_start = st_start,
                                                           st_len = st_len,
                                                           st_len_w2 = st_len_w2
                                                           )
    Purify_analysis.apply_sync_filter_w1_w2()
    corrected_corrs,u_corrected_corrs,no = Purify_analysis.correlate_RO_results_no_purification(apply_ROC = True,return_value = True,verbose=False)
    val00.append(corrected_corrs[3])
    val11.append(corrected_corrs[0])
    fidelities.append((corrected_corrs[1]+corrected_corrs[2] + 2*np.sqrt(TPQI_vis*corrected_corrs[1]*corrected_corrs[2]))/2)
    fidelities_u.append(np.sqrt((1+np.sqrt(TPQI_vis*corrected_corrs[2]/corrected_corrs[1]))*u_corrected_corrs[1]**2\
                               +(1+np.sqrt(TPQI_vis*corrected_corrs[1]/corrected_corrs[2]))*u_corrected_corrs[2]**2\
                               +np.sqrt(corrected_corrs[1]*corrected_corrs[2]/TPQI_vis)*TPQI_vis_u**2)/2)
    
    fidelities_mixed.append((corrected_corrs[1]+corrected_corrs[2])/2)
    fidelities_mixed_u.append(np.sqrt(u_corrected_corrs[1]**2+u_corrected_corrs[2]**2)/2)



In [ ]:
purified_state_pops = real_pop_vals[1:5]#np.array([15,25,34.5,50])/100.
## fidelities were derived from the ipython notebook Purification fidelities.
pur_Fids_m_and_p = np.flipud(np.array([[0.50086235948, 0.0291095025243,0.511807360737, 0.0293224710122],
                    [0.60207101112, 0.0343278921738,0.56564097942, 0.0340960124038],
                    [0.646418396751, 0.0333146903515,0.646120106595, 0.0343019299242],
                    [0.732227985693,  0.0392381744896, 0.72610488295, 0.0373220861641]]))


purified_state_fid =  0.5*(pur_Fids_m_and_p[:,0]+pur_Fids_m_and_p[:,2])
purified_state_fid_u =  0.5*np.sqrt(pur_Fids_m_and_p[:,1]**2+pur_Fids_m_and_p[:,3]**2)

model_theta_pur = [0.0954915, 0.146447, 0.206107, 0.273005, 0.345492, 0.421783, 0.5]
model_fids_pur = [0.715444, 0.701993, 0.68198, 0.656421, 0.625711, 0.590059,0.549683]

model_theta_store = [0.0244717, 0.0544967, 0.0954915, 0.146447, 0.206107, 0.273005, 
0.345492, 0.421783, 0.5, 0.578217, 0.654508]
model_fids_stored_0 = [0.719141, 0.756781, 0.746996, 0.716113, 0.672593, 0.620398, \
0.562094, 0.499757, 0.439788, 0.446315, 0.452703]
model_fids_stored_25 = [0.704976, 0.741284, 0.731845, 0.702055, 0.660076, 0.609729, \
0.553488, 0.493357, 0.435464, 0.441008, 0.446435]
model_fids_stored_50 = [0.678982, 0.712845, 0.704042, 0.676257, 0.637105, 0.590148, \
0.537695, 0.481613, 0.427528, 0.431269, 0.434931]

model_fids_e_0 = [0.737539, 0.777146, 0.76685, 0.734353, 0.688559, 0.633637, 0.572286, \
0.506691, 0.443709, 0.452459, 0.461024]
model_fids_e_25 = [0.737539, 0.777146, 0.76685, 0.734353, 0.688559, 0.633637, 0.572286, \
0.506691, 0.443709, 0.452459, 0.461024]
model_fids_e_50 = [0.737539, 0.777146, 0.76685, 0.734353, 0.688559, 0.633637, 0.572286, \
0.506691, 0.443709, 0.452459, 0.461024]

raw_estate_fids = [0.749274, 0.789511, 0.779051, 0.746036, 0.699514, 0.643718, \
0.581391, 0.514753, 0.450691, 0.458356, 0.465859]

def corrs_theta_theory(theta,pd1,pd2,pdc):
    p11 = 2*np.cos(theta)**4 * pdc
    p00=np.sin(theta)**4 * (pd1*(1-pd2)+pd2*(1-pd1))
    p01=np.sin(theta)**2 * np.cos(theta)**2 * pd2
    p10=np.sin(theta)**2 * np.cos(theta)**2 * pd1
    return np.array([p00,p01,p10,p11])/np.tile((p00+p01+p10+p11)[np.newaxis].T,4).T


pd1 = 8e-4;
pd2 = 0.5*pd1;
pdc = 2.5e-6;
e_state_thetas = np.arange(np.pi/20,np.pi/2+np.pi/21.,np.pi/20) 
e_state_pops = np.sin(e_state_thetas)**2
corrs_e_state = corrs_theta_theory(e_state_thetas,pd1,pd2,pdc)
model_fids_estate = ((corrs_e_state[1]+corrs_e_state[2] + 2*np.sqrt(TPQI_vis*corrs_e_state[1]*corrs_e_state[2]))/2)
model_fids_estate_mixed = ((corrs_e_state[1]+corrs_e_state[2])/2)

print TPQI_vis

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Helvetica'

In [ ]:
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*78
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)

lw = 1
fontsize = 10
markersize = 4
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)


plt.plot(model_theta_pur,model_fids_pur,linestyle='--',color='#3594F2',lw=lw)
plt.errorbar(purified_state_pops,purified_state_fid,purified_state_fid_u,marker='o',ms=markersize,mew =0.5,capsize=3, elinewidth=lw,ls='none', color='#3594F2',label = 'Purified state',zorder=10)
plt.plot(e_state_pops,model_fids_estate,linestyle='-',color='#F29335',lw=lw,zorder=0)

ax.set_ylabel('Fidelity w. Bell state')
plt.yticks([0.25,0.5,0.75])
plt.xlim([0.1,0.55])
plt.ylim([0.4,0.85])

### for the final plot we want to hide the x ticks as they are given in the panel below:

ax.set_xlabel(r'sin$^2 \theta$')

plt.savefig(os.path.join(save_folder,'did_we_purify_estate.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'did_we_purify_estate.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()
plt.close('all')

In [ ]:
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*78
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)

lw = 1
fontsize = 10
markersize = 4
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)

#-----
#colors
#-----
blue = '#3594F2'
orange = '#F29335'

espin_purple = '#632A85'
nspin_yellow = '#D2920B'

plt.errorbar(purified_state_pops,purified_state_fid,purified_state_fid_u,marker='o',
             ms=markersize,mew =0.5,capsize=3, elinewidth=lw,ls='none', color=blue,label = 'Distilled state',zorder=10)
plt.errorbar(real_pop_vals,fidelities_mixed,fidelities_mixed_u,marker='o',ms=markersize,mew =0.5,capsize=3,
             elinewidth=lw,ls='none',color=espin_purple,label = r'Measured raw'+"\n"+r'electron state',zorder = 10)
#r'Measured e$^-$state',zorder=10)
# plt.plot([0.0,1.0],[0.5,0.5],'-',color='black',alpha=0.5) # horizontal line for class. fidelity
ax.fill_between(model_theta_store,model_fids_stored_0,model_fids_stored_50,
                                    color=nspin_yellow,alpha=0.2,linewidth=0.0)
plt.plot(model_theta_store,model_fids_stored_25,linestyle='-',
                     color=nspin_yellow,lw=lw,label = 'Nuclei: raw state') #Stored raw state
# plt.plot(e_state_pops,model_fids_stored,linestyle='-.',color='black',lw=lw)
plt.plot(e_state_pops,model_fids_estate_mixed,linestyle='--',color=espin_purple,lw=lw,label = r'Model')
plt.plot(model_theta_pur,model_fids_pur,linestyle='--',color=blue,lw=lw,label = 'Model')

ax.fill_between(model_theta_store,model_fids_e_0,model_fids_e_50,color=espin_purple,alpha=0.2,linewidth=0.0)
plt.plot(model_theta_store,model_fids_e_25,linestyle='-',
         color=espin_purple,lw=lw,label = 'Electrons: raw state')#'e$^-$raw state')

print purified_state_pops
print model_theta_store
print model_fids_stored_0


#ax.set_ylabel('Fidelity w. Bell state')
ax.set_ylabel('State fidelity')
# linewidths = lw
# textsize = fontsize

plt.yticks([0.25,0.5,0.75])
plt.xticks([0.1,0.2,0.3,0.4,0.5])
plt.xlim([0.1,0.5])
plt.ylim([0.2,0.85])
# ax.legend()
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
labels,handles = list(labels),list(handles)
### get the right label ordering:
# labels[1],labels[2] = labels[2],labels[1]
# handles[1],handles[2] = handles[2],handles[1]

#### this assignment works for the old labels.
labels = [labels[0],labels[1],labels[5],labels[2]]
handles = [(handles[0],handles[-2]),handles[1],handles[5],(handles[-3],handles[2])]

# ax.legend()
ax.legend(handles, labels,borderaxespad=0.,fontsize =fontsize-1,frameon=False,
                  ncol=1,bbox_to_anchor=(1.01, 0.85), loc=2,numpoints=1,
                                  handletextpad = 0.2,columnspacing = 0.8)
# ax.legend(handles=[Fid,XX,YY,ZZ],ncol=2,numpoints = 1,frameon=False,
#                                   handlelength=0,handletextpad = 0.4,columnspacing = 0.8,
#                                   borderaxespad =0.05,
#                                   fontsize=fontsize-1)

### for the final plot we want to hide the x ticks as they are given in the panel below:
# ax.set_xticklabels([])
# ax.set_xlabel('Population in ms=0')
ax.set_xlabel(r'sin$^2 \theta$')

plt.savefig(os.path.join(save_folder,'did_we_purify_combined.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'did_we_purify_combined.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()
plt.close('all')

In [ ]:
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)

ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)


el_state_legend = r'Measured raw'+"\n"+r'electron state'
el_state_legend = 'Raw state: data'
plt.errorbar(purified_state_pops,purified_state_fid,purified_state_fid_u,marker='o',
             ms=markersize,mew =0.5,capsize=3, elinewidth=lw,ls='none', color=blue,label = 'Distilled state: data',zorder=10)
plt.errorbar(real_pop_vals,fidelities_mixed,fidelities_mixed_u,marker='o',ms=markersize,mew =0.5,capsize=3,
             elinewidth=lw,ls='none',color=espin_purple,label = el_state_legend,zorder = 10)

ax.fill_between(model_theta_store,model_fids_stored_0,model_fids_stored_50,
                                    color=nspin_yellow,alpha=0.2,linewidth=0.0)
plt.plot(model_theta_store,model_fids_stored_25,linestyle='-',
                     color=nspin_yellow,lw=lw,label = 'Nuclei: raw state') 

plt.plot(e_state_pops,model_fids_estate_mixed,linestyle='--',color=espin_purple,lw=lw,label = r'Raw state: model')
plt.plot(model_theta_pur,model_fids_pur,linestyle='--',color=blue,lw=lw,label = 'Distilled state: model')

# error bar for e model does not make sense yet. it is all the same right now
# ax.fill_between(model_theta_store,model_fids_e_0,model_fids_e_50,color=espin_purple,alpha=0.2,linewidth=0.0)
plt.plot(model_theta_store,model_fids_e_25,linestyle='-',
         color=espin_purple,lw=lw,label = 'Electrons: raw state')

ax.set_ylabel('State fidelity')


plt.yticks([0.25,0.5,0.75])
plt.xticks([0.1,0.2,0.3,0.4,0.5])
plt.xlim([0.1,0.5])
plt.ylim([0.2,0.85])

handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0])) #alphabetical sorting
labels,handles = list(labels),list(handles)


### we make an empty handle for descriptive purposes
r = mpl.patches.Rectangle((0,0), 1, 1, fill=False, edgecolor='none',
                                 visible=False)
#### this assignment works for the old labels.
labels = [labels[0],labels[1],labels[4],labels[5],"","Idealized models",labels[2],labels[3]]
handles = [handles[0],handles[1],handles[4],handles[5],r,r,handles[2],handles[3]]


ax.legend(handles, labels,borderaxespad=0.,fontsize =fontsize-1,frameon=False,
                  ncol=1,bbox_to_anchor=(1.01, 0.95), loc=2,numpoints=1,
                                  handletextpad = 0.2,columnspacing = 0.7,labelspacing = 0.25)

### for the final plot we want to hide the x ticks as they are given in the panel below:
ax.set_xlabel(r'sin$^2 \theta$')

plt.savefig(os.path.join(save_folder,'did_we_purify_combined_diff_labels.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'did_we_purify_combined_diff_labels.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()
plt.close('all')

In [ ]:
model_handles = handles[2]


In [ ]:
purple_line = handles[2][0]

## Hypothesis testing

Interpolate to find value at points

In [ ]:
nullValues = np.interp(purified_state_pops, model_theta_store, model_fids_e_25)
purState=zip(nullValues,purified_state_fid,purified_state_fid_u)

Error bars represent one standard deviation. Assume normally distributed outcomes (CLT)

In [ ]:
from scipy.stats import norm

for x, mean, error in purState:
    print norm.cdf(x,mean,error)
    

And vs raw state?

In [ ]:
nullValuesRaw = np.interp(purified_state_pops, model_theta_store, raw_estate_fids)
purStateVsRaw =zip(nullValuesRaw,purified_state_fid,purified_state_fid_u)

from scipy.stats import norm

for x, mean, error in purStateVsRaw:
    print norm.cdf(x,mean,error)


In [ ]:
purified_state_pops

In [ ]:
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*78
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)

lw = 1
fontsize = 10
markersize = 4
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)

plt.errorbar(purified_state_pops,purified_state_fid,purified_state_fid_u,marker='o',
             ms=markersize,mew =0.5,capsize=3, elinewidth=lw,ls='none', color='#3594F2',label = 'Distilled state',zorder=10)
plt.plot(model_theta_store,model_fids_e_25,linestyle='-',color='#50A53A',lw=lw,label = 'e$^-$raw state')
plt.plot(model_theta_store,raw_estate_fids,linestyle='--',color='#F29335',lw=lw,label = 'Initial state')


#ax.set_ylabel('Fidelity w. Bell state')
ax.set_ylabel('State fidelity')
# linewidths = lw
# textsize = fontsize

plt.yticks([0.25,0.5,0.75])
plt.xticks([0.1,0.2,0.3,0.4,0.5])
plt.xlim([0.1,0.5])
plt.ylim([0.4,0.8])
# ax.legend()
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
labels,handles = list(labels),list(handles)

# ax.legend()
ax.legend(handles, labels,borderaxespad=0.,fontsize =fontsize-1,frameon=False,
                  ncol=1,bbox_to_anchor=(1.01, 0.85), loc=2,numpoints=1,
                                  handletextpad = 0.2,columnspacing = 0.8)
# ax.legend(handles=[Fid,XX,YY,ZZ],ncol=2,numpoints = 1,frameon=False,
#                                   handlelength=0,handletextpad = 0.4,columnspacing = 0.8,
#                                   borderaxespad =0.05,
#                                   fontsize=fontsize-1)

### for the final plot we want to hide the x ticks as they are given in the panel below:
# ax.set_xticklabels([])
# ax.set_xlabel('Population in ms=0')
ax.set_xlabel(r'sin$^2 \theta$')

plt.savefig(os.path.join(save_folder,'pvaluePlot.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'pvaluePlot.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()
plt.close('all')